In [1]:
import os
import umap
import torch
import itertools
from core import *
import numpy as np
import pandas as pd
import pickle as pkl
from torch import nn
import os.path as op
from tqdm import tqdm
from time import time
from Bio import SeqIO
from copy import deepcopy
from torch.optim import Adam
from tpot import TPOTRegressor
import torch.nn.functional as F
import matplotlib.pyplot as plt
from capsules.capsules import *
from IPython.display import Image
from qhoptim.pyt import QHM, QHAdam
from matplotlib.lines import Line2D
from torch.autograd import Variable
from matplotlib.patches import Patch
from scipy.stats import spearmanr, pearsonr
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error
from sklearn.metrics import accuracy_score, matthews_corrcoef, precision_score, recall_score

/home/bakirillov/.local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/home/bakirillov/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/bakirillov/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selectio

In [11]:
len("TTTCTGAGCATCCCGGGGATCCC")

23

In [2]:
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

In [3]:
import gpytorch
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.models import ApproximateGP, ExactGP 
from sklearn.metrics import accuracy_score
from gpytorch.priors import SmoothedBoxPrior
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.kernels import ScaleKernel, RBFKernel, GridInterpolationKernel
from gpytorch.mlls import VariationalELBO, VariationalELBOEmpirical, DeepApproximateMLL
from gpytorch.distributions import MultivariateNormal
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP

In [4]:
DATAPATH = "/home/bakirillov/HDD/data/genomes/all_guides_chr22.tsv"
ANNOTATION = "/home/bakirillov/HDD/data/genomes/annotation.csv"

In [5]:
annotation = pd.read_csv(ANNOTATION, header=1)[["Start", "Stop", "Gene symbol"]]
annotation = list(
    zip(
        annotation["Start"].values, annotation["Stop"].values, 
        annotation["Gene symbol"].values
    )
)
annotation = sorted(annotation, key=lambda x: x[0])

In [6]:
transformer = get_Cas9_transformer(True)

In [7]:
chr22 = SeqIO.parse("/home/bakirillov/HDD/data/genomes/chr22.fa", "fasta")
chr22 = [a for a in chr22][0]

In [8]:
TEMPLATE = "/home/bakirillov/HDD/data/chr22_genes/GENE\nNNNNNNNNNNNNNNNNNNNNNGG\nNNNNNNNNNNNNNNNNNNNNNGG 0\n"
COMMAND = "/home/bakirillov/HDD/apps/cas-offinder CONFIG G OUTPUT"

In [9]:
script = open("run_casoffinder.sh", "w")
script.write("#!/bin/sh\n\n")
for start, stop, gene in tqdm(annotation):
    with open("/home/bakirillov/HDD/data/chr22_genes/"+gene+".fa", "w") as oh:
        oh.write(">"+gene+"\n")
        oh.write(str(chr22.seq[start:stop]))
    with open("/home/bakirillov/HDD/data/chr22_casoffinder_configs/"+gene+".txt", "w") as oh:
        oh.write(TEMPLATE.replace("GENE", gene+".fa"))
    script.write(
        COMMAND.replace(
            "CONFIG", "/home/bakirillov/HDD/data/chr22_casoffinder_configs/"+gene+".txt"
        ).replace(
            "OUTPUT", "/home/bakirillov/HDD/data/chr22_grnas/"+gene+".tsv"
        )+"\n"
    )
script.close()

100%|██████████| 1186/1186 [00:00<00:00, 2072.07it/s]


In [10]:
cnn_mse_encoder = GuideHN(21, 32, 1360, n_classes=5).cuda()
cnn_mse = DKL(cnn_mse_encoder, [1,5*32]).cuda()
cnn_mse.load_state_dict(torch.load("../models/DeepHF/Wt/cnn_mse59.ptch"))
cnn_mse = cnn_mse.eval()

In [11]:
rnn_mse_encoder = GuideHRNN(21, 32, 3360, n_classes=5).cuda()
rnn_mse = DKL(rnn_mse_encoder, [1,5*32]).cuda()
rnn_mse.load_state_dict(torch.load("../models/DeepHF/Wt/rnn_mse59.ptch"))
rnn_mse = rnn_mse.eval()

In [12]:
cnn_elbo_encoder = GuideHN(21, 32, 1360, n_classes=5).cuda()
cnn_elbo = DKL(cnn_elbo_encoder, [1,5*32]).cuda()
cnn_elbo.load_state_dict(torch.load("../models/DeepHF/Wt/cnn_elbo59.ptch"))
cnn_elbo = cnn_elbo.eval()

In [13]:
rnn_elbo_encoder = GuideHRNN(21, 32, 3360, n_classes=5).cuda()
rnn_elbo = DKL(rnn_elbo_encoder, [1,5*32]).cuda()
rnn_elbo.load_state_dict(torch.load("../models/DeepHF/Wt/rnn_elbo59.ptch"))
rnn_elbo = rnn_elbo.eval()

In [14]:
data = np.array(
    [a for a in os.walk("/home/bakirillov/HDD/data/chr22_grnas/")][0][2]
)

In [15]:
sizes = np.array(
    [op.getsize(op.join("/home/bakirillov/HDD/data/chr22_grnas", a)) for a in data]
)

In [20]:
current_df[3].apply(len).unique()

array([23, 18, 20, 19,  9,  8,  7])

In [21]:
genes = {a[2]: [] for a in annotation}
for i,a in enumerate(data):
    #with open(op.join("/home/bakirillov/HDD/data/casoffinder_data/", a), "r"):
    current_df = pd.read_csv(
        op.join(
            "/home/bakirillov/HDD/data/chr22_grnas/"+a
        ), sep="\t", header=None
    ).dropna()
    current_df = current_df[current_df[3].apply(lambda x: len(x) == 23)]
    current_df[6] = [a]*current_df.shape[0]
    print(i, current_df.shape)
    tds = DeepHFDataset(
        current_df, np.arange(current_df.shape[0]), transformer, sequence_column=3, 
        label_column=5
    )
    tld = DataLoader(tds, shuffle=False, batch_size=256)
    tb_cnn_mse_oa = []
    tb_cnn_mse_va = []
    tb_rnn_mse_oa = []
    tb_rnn_mse_va = []
    tb_cnn_elbo_oa = []
    tb_cnn_elbo_va = []
    tb_rnn_elbo_oa = []
    tb_rnn_elbo_va = []
    for transformed_batch, _ in tqdm(tld):
        tb_cnn_mse = cnn_mse(transformed_batch)
        tb_cnn_mse_o = cnn_mse.likelihood(
            tb_cnn_mse[0]
        ).mean.mean(0).cpu().data.numpy()
        tb_cnn_mse_v = cnn_mse.likelihood(
            tb_cnn_mse[0]
        ).variance.mean(0).cpu().data.numpy()
        tb_cnn_mse_oa.extend(tb_cnn_mse_o)
        tb_cnn_mse_va.extend(tb_cnn_mse_v)
        tb_rnn_mse = rnn_mse(transformed_batch)
        tb_rnn_mse_o = rnn_mse.likelihood(
            tb_rnn_mse[0]
        ).mean.mean(0).cpu().data.numpy()
        tb_rnn_mse_v = rnn_mse.likelihood(
            tb_rnn_mse[0]
        ).variance.mean(0).cpu().data.numpy()
        tb_rnn_mse_oa.extend(tb_rnn_mse_o)
        tb_rnn_mse_va.extend(tb_rnn_mse_v)
        tb_cnn_elbo = cnn_elbo(transformed_batch)
        tb_cnn_elbo_o = cnn_elbo.likelihood(
            tb_cnn_elbo[0]
        ).mean.mean(0).cpu().data.numpy()
        tb_cnn_elbo_v = cnn_elbo.likelihood(
            tb_cnn_elbo[0]
        ).variance.mean(0).cpu().data.numpy()
        tb_cnn_elbo_oa.extend(tb_cnn_elbo_o)
        tb_cnn_elbo_va.extend(tb_cnn_elbo_v)
        tb_rnn_elbo = rnn_elbo(transformed_batch)
        tb_rnn_elbo_o = rnn_elbo.likelihood(
            tb_rnn_elbo[0]
        ).mean.mean(0).cpu().data.numpy()
        tb_rnn_elbo_v = rnn_elbo.likelihood(
            tb_rnn_elbo[0]
        ).variance.mean(0).cpu().data.numpy()
        tb_rnn_elbo_oa.extend(tb_rnn_elbo_o)
        tb_rnn_elbo_va.extend(tb_rnn_elbo_v)
    current_df[7] = tb_cnn_mse_oa
    current_df[8] = tb_cnn_mse_va
    current_df[9] = tb_rnn_mse_oa
    current_df[10] = tb_rnn_mse_va
    current_df[11] = tb_cnn_elbo_oa
    current_df[12] = tb_cnn_elbo_va
    current_df[13] = tb_rnn_elbo_oa
    current_df[14] = tb_rnn_elbo_va
    current_df[15] = (current_df[7] + current_df[9])/2
    current_df[16] = current_df[8] + current_df[10]
    current_df[17] = (current_df[11] + current_df[13])/2
    current_df[18] = current_df[12] + current_df[14]
    current_df = current_df.sort_values(by=7, ascending=False)
    current_df.to_csv(
        op.join(
            "/home/bakirillov/HDD/data/chr22_ordered/"+a
        ), sep="\t", header=None
    )

 14%|█▍        | 1/7 [00:00<00:00,  6.72it/s]

0 (1743, 7)


  0%|          | 0/1 [00:00<?, ?it/s]

1 (116, 7)
2 (123, 7)


  7%|▋         | 1/15 [00:00<00:02,  6.55it/s]

...

  7%|▋         | 1/14 [00:00<00:01,  7.11it/s]

1184 (3465, 7)


 20%|██        | 1/5 [00:00<00:00,  6.85it/s]

1185 (1082, 7)


100%|██████████| 5/5 [00:00<00:00,  7.72it/s]


In [22]:
ordered_data = [a for a in os.walk("/home/bakirillov/HDD/data/chr22_ordered/")][0][2]

In [38]:
total = None
nothing = []
for i,a in enumerate(ordered_data):
    print(i,a)
    try:
        current_df = pd.read_csv(
            op.join(
                "/home/bakirillov/HDD/data/chr22_ordered/"+a
            ), sep="\t", header=None
        )
    except:
        nothing.append(a)
        print(i,a, "nothing")
    else:
        if i == 0:
            total = current_df.head(10)
        else:
            total = total.append(current_df.head(10))

0 SERPIND1.tsv
1 HSFY1P1.tsv
2 SLC25A15P5.tsv
3 LOC339685.tsv
4 GTSE1-DT.tsv
5 TPTEP2.tsv
6 RNU6-1219P.tsv
7 SAMM50.tsv
8 PLA2G6.tsv
9 XPNPEP3.tsv
10 LOC105373007.tsv
11 RPL34P35.tsv
12 APOBEC3A.tsv
13 LINC00896.tsv
14 CHKB.tsv
15 TMEM184B.tsv
16 MIR4763.tsv
17 DGCR6.tsv
18 LOC105372860.tsv
19 SEZ6L.tsv
20 RPL32P5.tsv
21 CA15P1.tsv
22 LINC00634.tsv
23 MIR6069.tsv
24 CRELD2.tsv
25 HSCB.tsv
26 LOC101232810.tsv
27 APOBEC3C.tsv
28 RPL15P22.tsv
29 LOC730668.tsv
30 CACNA1I.tsv
31 MEI1.tsv
32 LOC107985568.tsv
33 HIC2.tsv
34 CRIP1P4.tsv
35 MYH9.tsv
36 RASD2.tsv
37 IGLV2-14.tsv
38 LOC101927447.tsv
39 TOM1.tsv
40 LOC105373067.tsv
41 RNA5SP493.tsv
42 VN1R9P.tsv
43 LOC105373015.tsv
44 GGTLC2.tsv
45 LOC105373018.tsv
46 CDC45.tsv
47 ANP32BP2.tsv
48 RTN4R.tsv
49 SHANK3.tsv
50 SULT4A1.tsv
51 IGLJ7.tsv
51 IGLJ7.tsv nothing
52 ZNF73P.tsv
53 KRT18P5.tsv
54 RPL8P5.tsv
55 RN7SL168P.tsv
56 LINC01639.tsv
57 TPTEP1.tsv
58 LOC101927474.tsv
59 SREBF2-AS1.tsv
60 MIR6889.tsv
61 PCMTD1P4.tsv
62 SEC14L2.tsv
63 RPS1

503 IGLL3P.tsv
504 MIR3201.tsv
505 RPS18P14.tsv
506 MIR648.tsv
507 BRD1.tsv
508 HMOX1.tsv
509 PDXP.tsv
510 MIR1286.tsv
511 ARFGAP3.tsv
512 IGLV3-31.tsv
513 RPL5P35.tsv
514 MORC2.tsv
515 LOC105372990.tsv
516 LOC101929638.tsv
517 TTC28.tsv
518 CABIN1.tsv
519 PISD.tsv
520 ABHD17AP5.tsv
521 IGLV2-11.tsv
522 RN7SL20P.tsv
523 SUN2.tsv
524 TOP1P2.tsv
525 MTATP6P20.tsv
526 LOC100996404.tsv
527 RN7SL843P.tsv
528 LOC105372997.tsv
529 SLC25A17.tsv
530 SNORA77B.tsv
531 LOC105373012.tsv
532 IGLV3-16.tsv
533 IGLV3-32.tsv
534 IGLVI-68.tsv
535 LOC110467534.tsv
536 RGL4.tsv
537 MIR4761.tsv
538 CYTH4.tsv
539 LOC284930.tsv
540 MIR6817.tsv
541 LOC100419811.tsv
542 IGLVIV-66-1.tsv
543 UQCRFS1P1.tsv
544 MIR185.tsv
545 LOC105372979.tsv
546 ZNF72P.tsv
547 IGLVV-66.tsv
548 GSTT2.tsv
549 IGLV3-21.tsv
550 LOC100421121.tsv
551 GALR3.tsv
552 IGLV3-6.tsv
553 BCRP6.tsv
554 SNORD125.tsv
555 LOC101929255.tsv
556 POM121L10P.tsv
557 LOC107985577.tsv
558 BCRP3.tsv
559 IGLV11-55.tsv
560 PPM1F-AS1.tsv
561 RNA5SP496.tsv
562

991 LOC107985557.tsv
992 LOC105372974.tsv
993 PDGFB.tsv
994 RNU6-338P.tsv
995 SHISAL1.tsv
996 GSTT4.tsv
997 LOC391334.tsv
998 MRTFA-AS1.tsv
999 PPP1R14BP1.tsv
1000 ATP6V1E1.tsv
1001 LOC102724801.tsv
1002 LOC105373059.tsv
1003 LOC112268289.tsv
1004 RN7SKP169.tsv
1005 LOC105372977.tsv
1006 WNT7B.tsv
1007 RPS15AP37.tsv
1008 IGLVVII-41-1.tsv
1009 ACO2.tsv
1010 LOC107985576.tsv
1011 RRP7A.tsv
1012 SLC5A1.tsv
1013 MAPK11.tsv
1014 MAPK8IP2.tsv
1015 SLC2A11.tsv
1016 LOC644525.tsv
1017 PANX2.tsv
1018 TPST2.tsv
1019 LOC100113375.tsv
1020 NAGA.tsv
1021 TMPRSS6.tsv
1022 RNU6-375P.tsv
1023 VPREB3.tsv
1024 LOC105372853.tsv
1025 RN7SL305P.tsv
1026 APOL5.tsv
1027 LOC105372995.tsv
1028 LOC105372858.tsv
1029 LOC101927344.tsv
1030 CABP7.tsv
1031 TBC1D22A.tsv
1032 LOC105373063.tsv
1033 ESS2.tsv
1034 POM121L1P.tsv
1035 AIFM3.tsv
1036 CA15P3.tsv
1037 OR11H1.tsv
1038 LOC100420885.tsv
1039 HMGN2P10.tsv
1040 LOC107985537.tsv
1041 MIR4766.tsv
1042 SGSM3.tsv
1043 ABCD1P4.tsv
1044 CCT8L2.tsv
1045 IGLC4.tsv
1046 G

In [39]:
nothing

['IGLJ7.tsv', 'IGLJ3.tsv', 'IGLJ4.tsv', 'IGLJ2.tsv', 'IGLJ5.tsv']

In [35]:
del total[0]
del total[1]
del total[6]
del total[7]

In [42]:
np.unique(total[5].values, return_counts=True)

(array(['+', '-'], dtype=object), array([5622, 5931]))

In [37]:
total.to_csv("DeepHF_wt_paper_ready.tsv")